In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re
import requests
import spacy
import nltk
nlp = spacy.load("en_core_web_sm")
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize

from collections import Counter
from PIL import Image
from io import BytesIO
from transformers import BertModel, BertTokenizer

from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, PCA
from sklearn.preprocessing import StandardScaler

from keras.applications.vgg16 import VGG16

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/wtyjoycee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/wtyjoycee/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/wtyjoycee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/wtyjoycee/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [11]:
product_data_cluster = pd.read_csv("product_data_cluster.csv")
product_data_cluster.head(2)

,Unnamed: 0,Category,ID,Product Name,Price,Shipping Location,Product Description,Store Name,Store Link,Product Link,Product Image,processed_Name,Nouns,Labels,functional_nouns,Labels_functional,Cluster,Cluster_Name
0,0,Health protective gear,733229857662,Cotton knee warm old cold legs men and women j...,5.94,"Hangzhou, Zhejiang",Free shipping free factory shipping,Daily sale factory shop,'https://store.taobao.com/shop/view_shop.htm?a...,'https://item.taobao.com/item.htm?id=733229857662,'https://g-search2.alicdn.com/img/bao/uploaded...,cotton knee warm old cold leg men woman joint ...,"['cotton', 'knee', 'leg', 'men', 'woman', 'joi...",cotton,"['cotton', 'knee', 'leg', 'men', 'woman', 'sum...",cotton,71,"cold, knee"
1,1,fitness,742827915487,Healthy belly wheel automatic rebound abdomina...,20.72,"Hangzhou, Zhejiang",Free shipping from Amoy factory,Daily sale factory shop,'https://store.taobao.com/shop/view_shop.htm?a...,'https://item.taobao.com/item.htm?id=742827915487,'https://g-search2.alicdn.com/img/bao/uploaded...,healthy belly wheel automatic rebound abdomina...,"['rebound', 'retraction', 'exercise', 'men', '...",rebound,"['belly', 'wheel', 'retraction', 'exercise', '...",belly,25,"abdominal, fitness"


# TF-IDF COS-Similarity Visualization

In [131]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(product_data_cluster['processed_Name'].astype(str).tolist())

In [ ]:
clusters

In [132]:
num_clusters = 150  
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(30, 20))  
ax = sns.heatmap(category_cluster_matrix, annot=False, cmap="YlGnBu")

ax.set_title("Heatmap of Categories vs Clusters")
ax.set_xlabel("Clusters")
ax.set_ylabel("Categories")

cluster_labels = [f"{i}" for i in pivot_table.columns]
ax.set_xticks(range(len(cluster_labels)))
ax.set_xticklabels(cluster_labels, rotation=90)

category_labels = pivot_table.index.tolist()
ax.set_yticks(range(len(category_labels)))
ax.set_yticklabels(category_labels, rotation=0)

plt.show()


# TF-IDF Silhouette Score Visualization

In [ ]:
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.sparse import csr_matrix, isspmatrix_csr

if isspmatrix_csr(tfidf_matrix):
    # Convert to a dense matrix to handle NaNs and Infs
    tfidf_matrix = tfidf_matrix.todense()

# Check for NaN and infinite values
if np.any(np.isnan(tfidf_matrix)) or np.any(np.isinf(tfidf_matrix)):
    print("NaN or Inf values found in tfidf_matrix. Replacing with 0.")
    tfidf_matrix = np.nan_to_num(tfidf_matrix)
    
num_clusters = sorted(product_data_cluster['Cluster'].unique()) 
silhouette_scores = []

for k in range(0,150):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(tfidf_matrix)
    labels = kmeans.labels_
    score = silhouette_score(tfidf_matrix, labels)
    silhouette_scores.append(score)

# Plotting
plt.plot(num_clusters, silhouette_scores, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score for Different Number of Clusters')
plt.show()


/Users/wtyjoycee/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [15]:
# # Group our data by cluster
# grouped = product_data_cluster.groupby('Cluster')

# # Initialize a DataFrame to hold the lowest priced products for each cluster
# lowest_priced_products = pd.DataFrame(columns=product_data_cluster.columns)

# # Iterate over each cluster group and find the product with the lowest price
# for cluster_id, group in grouped:
#     lowest_priced_product = group.loc[group['Price'].idxmin()]
#     lowest_priced_products = lowest_priced_products.append(lowest_priced_product)

# # Reset index for the new DataFrame
# lowest_priced_products.reset_index(drop=True, inplace=True)

# # Now, for a given product, you can recommend a lower-priced alternative from the same cluster
# def recommend_cheaper_alternatives(product_row, lowest_priced_products):
#     # Find the cluster of the input product
#     product_cluster = product_row['Cluster']
    
#     # Get the lowest priced product in the same cluster
#     recommendation = lowest_priced_products[lowest_priced_products['Cluster'] == product_cluster]
    
#     # Check if the recommended product is cheaper than the input product
#     if not recommendation.empty and recommendation.iloc[0]['Price'] < product_row['Price']:
#         return recommendation.iloc[0]  # Return the recommended product row
#     else:
#         return None  # If no cheaper alternative is found, return None

# # Example usage: Let's assume you have a product_row that you want to find a cheaper alternative for
# # product_row should be a row from your product_data_cluster DataFrame
# product_row = product_data_cluster.iloc[0]

# # Call the function with the product_row
# recommendation = recommend_cheaper_alternatives(product_row, lowest_priced_products)

# # If a recommendation is found, you can process it further
# if recommendation is not None:
#     print(f"We recommend this cheaper alternative: {recommendation['Product Name']} with price {recommendation['Price']}")
# else:
#     print("No cheaper alternative found within the same cluster.")


/var/folders/00/rmfcmjw917ddlkkpky35rh380000gn/T/ipykernel_42522/2747462767.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  lowest_priced_products = lowest_priced_products.append(lowest_priced_product)
/var/folders/00/rmfcmjw917ddlkkpky35rh380000gn/T/ipykernel_42522/2747462767.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  lowest_priced_products = lowest_priced_products.append(lowest_priced_product)
/var/folders/00/rmfcmjw917ddlkkpky35rh380000gn/T/ipykernel_42522/2747462767.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  lowest_priced_products = lowest_priced_products.append(lowest_priced_product)
/var/folders/00/rmfcmjw917ddlkkpky35rh380000gn/T/ipykernel_42522/2747462767.py:10: FutureWarning: The frame.appe

/var/folders/00/rmfcmjw917ddlkkpky35rh380000gn/T/ipykernel_42522/2747462767.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  lowest_priced_products = lowest_priced_products.append(lowest_priced_product)
/var/folders/00/rmfcmjw917ddlkkpky35rh380000gn/T/ipykernel_42522/2747462767.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  lowest_priced_products = lowest_priced_products.append(lowest_priced_product)
/var/folders/00/rmfcmjw917ddlkkpky35rh380000gn/T/ipykernel_42522/2747462767.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  lowest_priced_products = lowest_priced_products.append(lowest_priced_product)
/var/folders/00/rmfcmjw917ddlkkpky35rh380000gn/T/ipykernel_42522/2747462767.py:10: FutureWarning: The frame.appe

We recommend this cheaper alternative: Knee pads/Warm old cold legs knee sheathing elderly men and women joint air conditioning room summer magic ultra-thin cold protection with price 2.0


/var/folders/00/rmfcmjw917ddlkkpky35rh380000gn/T/ipykernel_42522/2747462767.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  lowest_priced_products = lowest_priced_products.append(lowest_priced_product)
/var/folders/00/rmfcmjw917ddlkkpky35rh380000gn/T/ipykernel_42522/2747462767.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  lowest_priced_products = lowest_priced_products.append(lowest_priced_product)
/var/folders/00/rmfcmjw917ddlkkpky35rh380000gn/T/ipykernel_42522/2747462767.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  lowest_priced_products = lowest_priced_products.append(lowest_priced_product)
/var/folders/00/rmfcmjw917ddlkkpky35rh380000gn/T/ipykernel_42522/2747462767.py:10: FutureWarning: The frame.appe

In [19]:
product_data_cluster.columns

Index(['Unnamed: 0', 'Category', 'ID', 'Product Name', 'Price',
       'Shipping Location', 'Product Description', 'Store Name', 'Store Link',
       'Product Link', 'Product Image', 'processed_Name', 'Nouns', 'Labels',
       'functional_nouns', 'Labels_functional', 'Cluster', 'Cluster_Name'],
      dtype='object')

In [21]:

df = product_data_cluster  # our data with our picked clusters

# Group by 'Cluster' and find the product with the lowest price in each cluster
lowest_priced_products = df.loc[df.groupby('Cluster')['Price'].idxmin()]

def search_for_products(query, data):
    matching_products = data[data['Product Name'].str.contains(query, case=False, na=False)]
    return matching_products

def get_recommendations(product, all_products, lowest_priced_products):
    cluster = product['Cluster']
    # Filter out the current product and other clusters
    cluster_products = all_products[(all_products['Cluster'] == cluster) & (all_products['ID'] != product['ID'])]
    
    if not cluster_products.empty:
        # Get the lowest-priced product in this cluster
        lowest_price_product = lowest_priced_products[lowest_priced_products['Cluster'] == cluster]
        if not lowest_price_product.empty:
            # Return a Series with the recommended product information
            recommended_product = lowest_price_product.iloc[0]
            return pd.Series({
                'Recommended Product Name': recommended_product['Product Name'],
                'Recommended Price': recommended_product['Price'],
                'Recommended Product Link': recommended_product['Product Link']
            })
    # Return None or pd.Series with NaN if no recommendation is found
    return pd.Series({
        'Recommended Product Name': None,
        'Recommended Price': None,
        'Recommended Product Link': None
    })

# Apply the get_recommendations function to each row in the search results
recommendations = search_results.apply(
    lambda row: get_recommendations(row, df, lowest_priced_products), axis=1
)

# Join the recommendations with the search results
search_results = search_results.join(recommendations)


In [26]:

def get_multiple_recommendations(product, all_products, lowest_priced_products, num_recommendations=5):
    cluster = product['Cluster']
    cluster_products = all_products[(all_products['Cluster'] == cluster) & (all_products['ID'] != product['ID'])]
    
    if not cluster_products.empty:
        # pick the top num_recommendations
        recommended_products = cluster_products.nsmallest(num_recommendations, 'Price')
        return recommended_products[['Product Name', 'Price', 'Product Link']]
    return pd.DataFrame()


In [24]:
# find recommendation products for each category within each cluster

df = product_data_cluster  

# Function to find recommendations within the same cluster
def find_recommendations_for_cluster(cluster_df):
    # Sort the products in the cluster by price
    sorted_cluster_df = cluster_df.sort_values(by='Price')
    
    sorted_cluster_df['Recommended Product'] = None
    sorted_cluster_df['Recommended Price'] = None

    for i, product in sorted_cluster_df.iterrows():
        cheaper_products = sorted_cluster_df[sorted_cluster_df['Price'] < product['Price']]
        if not cheaper_products.empty:
            recommended_product = cheaper_products.iloc[0]
            sorted_cluster_df.at[i, 'Recommended Product'] = recommended_product['Product Name']
            sorted_cluster_df.at[i, 'Recommended Price'] = recommended_product['Price']

    return sorted_cluster_df

# Group by cluster and apply the recommendation function
grouped = df.groupby('Cluster')
recommendation_results = pd.concat([find_recommendations_for_cluster(group) for _, group in grouped])

In [27]:
df['Recommendations'] = df.apply(lambda row: get_multiple_recommendations(row, df, lowest_priced_products), axis=1)


In [29]:
df

,Unnamed: 0,Category,ID,Product Name,Price,Shipping Location,Product Description,Store Name,Store Link,Product Link,Product Image,processed_Name,Nouns,Labels,functional_nouns,Labels_functional,Cluster,Cluster_Name,Recommendations
0,0,Health protective gear,733229857662,Cotton knee warm old cold legs men and women j...,5.94,"Hangzhou, Zhejiang",Free shipping free factory shipping,Daily sale factory shop,'https://store.taobao.com/shop/view_shop.htm?a...,'https://item.taobao.com/item.htm?id=733229857662,'https://g-search2.alicdn.com/img/bao/uploaded...,cotton knee warm old cold leg men woman joint ...,"['cotton', 'knee', 'leg', 'men', 'woman', 'joi...",cotton,"['cotton', 'knee', 'leg', 'men', 'woman', 'sum...",cotton,71,"cold, knee",Pr...
1,1,fitness,742827915487,Healthy belly wheel automatic rebound abdomina...,20.72,"Hangzhou, Zhejiang",Free shipping from Amoy factory,Daily sale factory shop,'https://store.taobao.com/shop/view_shop.htm?a...,'https://item.taobao.com/item.htm?id=742827915487,'https://g-search2.alicdn.com/img/bao/uploaded...,healthy belly wheel automatic rebound abdomina...,"['rebound', 'retraction', 'exercise', 'men', '...",rebound,"['belly', 'wheel', 'retraction', 'exercise', '...",belly,25,"abdominal, fitness",Pr...
2,2,fitness,694022432481,361 yoga mat floor mat home yoga fitness mat f...,38.00,"Hangzhou, Zhejiang",Free shipping covered,361 degrees Hangzhou monopoly,'https://store.taobao.com/shop/view_shop.htm?a...,'https://item.taobao.com/item.htm?id=694022432481,'https://g-search2.alicdn.com/img/bao/uploaded...,yoga mat floor mat home yoga fitness mat men w...,"['mat', 'floor', 'home', 'yoga', 'fitness', 'm...",mat,"['yoga', 'mat', 'floor', 'mat', 'home', 'yoga'...",yoga,82,"mat, floor",Pr...
3,3,Children's furniture,743578283255,La Chapelle girls nightwear autumn and winter ...,69.90,"Yichang, Hubei Province",Free shipping for new products,lachapelle Vorhua Boutique,'https://store.taobao.com/shop/view_shop.htm?a...,'https://item.taobao.com/item.htm?id=743578283255,'https://gw.alicdn.com/imgextra/O1CN01D1PC2B1T...,la chapelle girl nightwear autumn winter new c...,"['la', 'autumn', 'winter', 'childrens', 'winte...",autumn,"['chapelle', 'girl', 'nightwear', 'autumn', 'w...",chapelle,66,"pajama, boy",Pr...
4,4,Other lamps,744708076380,led socket lamp head with switch two or three ...,3.50,"Maoming, Guangdong",Exempt from postage,International electrician brand promotion store,'https://store.taobao.com/shop/view_shop.htm?a...,'https://item.taobao.com/item.htm?id=744708076380,'https://g-search1.alicdn.com/img/bao/uploaded...,led socket lamp head switch two three pin ener...,"['socket', 'lamp', 'head', 'pin', 'plug', 'lam...",socket,"['socket', 'lamp', 'head', 'pin', 'lighting', ...",socket,68,"bulb, screw",Pr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89369,89369,Table setting,653453063908,Portable tableware Stainless steel chopsticks ...,2.80,"Guangzhou, Guangdong",Exempt from postage,Hee hee home daily department store,'https://store.taobao.com/shop/view_shop.htm?a...,'https://item.taobao.com/item.htm?id=653453063908,'https://g-search1.alicdn.com/img/bao/uploaded...,portable tableware stainless steel chopstick s...,"['tableware', 'stainless', 'steel', 'spoon', '...",tableware,"['steel', 'chopstick', 'spoon', 'threepiece', ...",steel,18,"chopstick, tableware",Pr...
89370,89370,Table setting,645088390464,Japanese ramen bowl noodle house dedicated to ...,16.80,"Jingdezhen, Jiangxi Province",Exempt from postage,yomerto flagship store,'https://store.taobao.com/shop/view_shop.htm?a...,'https://item.taobao.com/item.htm?id=645088390464,'https://g-search1.alicdn.com/img/bao/uploaded...,japanese ramen bowl noodle house dedicated eat...,"['ramen', 'house', 'bowl', 'household', 'soup'...",ramen,"['japanese', 'ramen', 'bowl', 'noodle', 'house...",japanese,97,"bowl, food",Pr...
89371,89371,Table fabric,631678780616,Nordic ins wind tpu tasteless table cloth clot...,29.00,Zhejiang Jinhua

# Price Analysis

In [44]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.io import push_notebook, output_notebook

output_notebook()


def create_cluster_plot(cluster_id):
    cluster_data = df[df['Cluster'] == cluster_id]

    if cluster_data.empty:
        print(f"No data found for cluster {cluster_id}")
        return None

    source_data = {
        'Price': cluster_data['Price'].tolist(),
        'Category': cluster_data['Category'].tolist()
    }

    source = ColumnDataSource(data=source_data)

    # Create the plot
    plot = figure(plot_width=600, plot_height=400, title=f"Cluster {cluster_id} - Price vs Category",
                  x_range=(cluster_data['Price'].min(), cluster_data['Price'].max()),
                  y_range=sorted(cluster_data['Category'].unique()))

    plot.scatter(x='Price', y='Category', source=source, size=8)
    plot.xaxis.axis_label = "Price"
    plot.yaxis.axis_label = "Category"

    return plot

# dropdown
def update_plot(change):
    clear_output(wait=True)
    cluster_id = int(change.new)
    plot = create_cluster_plot(cluster_id)
    if plot:
        handle = show(plot, notebook_handle=True)
        push_notebook(handle=handle)

# Dropdown widget
dropdown = widgets.Dropdown(options=[str(i) for i in range(150)], description='Select Cluster:')
dropdown.observe(update_plot, names='value')

# default plot
initial_cluster_id = 0
initial_plot = create_cluster_plot(initial_cluster_id)
if initial_plot:
    handle = show(initial_plot, notebook_handle=True)

# Display dropdown
display(dropdown)


In [88]:
df[df['Cluster'] == 0]

,Unnamed: 0,Category,ID,Product Name,Price,Shipping Location,Product Description,Store Name,Store Link,Product Link,...,processed_Name,Nouns,Labels,functional_nouns,Labels_functional,Cluster,Cluster_Name,Recommendations,Product Image Cleaned,Product Link Cleaned
147,147,Motorcycle rider equipment,743079636790,Electric motorcycle cold pants knee guard wint...,18.80,"Hangzhou, Zhejiang",Free shipping for new products,Bimei flagship store,'https://store.taobao.com/shop/view_shop.htm?a...,'https://item.taobao.com/item.htm?id=743079636790,...,electric motorcycle cold pant knee guard winte...,"['motorcycle', 'cold', 'pant', 'knee', 'guard'...",motorcycle,"['motorcycle', 'pant', 'knee', 'guard', 'winte...",motorcycle,0,"knee, motorcycle",Pr...,https://g-search3.alicdn.com/img/bao/uploaded/...,https://item.taobao.com/item.htm?id=743079636790
347,347,Motorcycle rider equipment,741808794406,Knee pads for electric vehicles Winter motorcy...,19.80,Zhejiang Jinhua,Exempt from postage,Mei Qian car products franchise shop,'https://store.taobao.com/shop/view_shop.htm?a...,'https://item.taobao.com/item.htm?id=741808794406,...,knee pad electric vehicle winter motorcycle ri...,"['knee', 'pad', 'vehicle', 'winter', 'motorcyc...",knee,"['knee', 'pad', 'electric', 'vehicle', 'winter...",knee,0,"knee, motorcycle",Pr...,https://g-search2.alicdn.com/img/bao/uploaded/...,https://item.taobao.com/item.htm?id=741808794406
837,837,Motorcycle rider equipment,692763337198,Leather knee pads keep warm for electric bikes...,10.90,"Yantai, Shandong Province",Exempt from postage,Yufei sports flagship store,'https://store.taobao.com/shop/view_shop.htm?a...,'https://item.taobao.com/item.htm?id=692763337198,...,leather knee pad keep warm electric bike cycli...,"['knee', 'pad', 'bike', 'cycling', 'gear', 'mo...",knee,"['leather', 'knee', 'pad', 'bike', 'cycling', ...",leather,0,"knee, motorcycle",Pr...,https://g-search1.alicdn.com/img/bao/uploaded/...,https://item.taobao.com/item.htm?id=692763337198
1029,1029,Motorcycle rider equipment,598612011959,Autumn and winter motorcycle stainless steel k...,26.60,Shanghai,Free shipping insurance free shipping baby,Volley Car shop,'https://store.taobao.com/shop/view_shop.htm?a...,'https://item.taobao.com/item.htm?id=598612011959,...,autumn winter motorcycle stainless steel knee ...,"['autumn', 'winter', 'motorcycle', 'stainless'...",autumn,"['autumn', 'winter', 'motorcycle', 'steel', 'k...",autumn,0,"knee, motorcycle",Pr...,https://g-search1.alicdn.com/img/bao/uploaded/...,https://item.taobao.com/item.htm?id=598612011959
1228,1228,Motorcycle rider equipment,740997798703,Motorcycle knee protector warm cycling winter ...,16.80,"Nanjing, Jiangsu",Exempt from postage,High quality thermal products factory,'https://store.taobao.com/shop/view_shop.htm?a...,'https://item.taobao.com/item.htm?id=740997798703,...,motorcycle knee protector warm cycling winter ...,"['motorcycle', 'knee', 'protector', 'warm', 'c...",motorcycle,"['knee', 'protector', 'cycling', 'winter', 'cy...",knee,0,"knee, motorcycle",Pr...,https://g-search1.alicdn.com/img/bao/uploaded/...,https://item.taobao.com/item.htm?id=740997798703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87878,87878,Motorcycle rider equipment,739842300490,Electric car knee pads warm winter riding moto...,10.00,"Langfang, Hebei",Free delivery of charity baby,Home furnishing store with knee pads,'https://store.taobao.com/shop/view_shop.htm?a...,'https://item.taobao.com/item.htm?id=739842300490,...,electric car knee pad warm winter riding motor...,"['car', 'knee', 'pad', 'winter', 'motorcycle',...",car,"['car', 'knee', 'pad', 'winter', 'motorcycle',...",car,0,"knee, motorcycle",Pr...,https://g-search2.alicdn.com/img/bao/uploaded/...,https://item.taobao.com/item.htm?id=739842300490
88042,88042,Motorcycle rider equipment,626786459747,Short knee pads for electric vehicles Winter m...,39.00,"Lishui, Zhejiang",Free shipping insurance free shipping baby,Dodo Sun

In [141]:
from bokeh.models import HoverTool
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.io import push_notebook
import ipywidgets as widgets
from IPython.display import display, clear_output

output_notebook()


def create_cluster_plot(cluster_id, df):
    cluster_data = df[df['Cluster'] == cluster_id]
    cluster_name = df['Cluster_Name'][df['Cluster'] == cluster_id]
    
    if cluster_data.empty:
        print(f"No data found for cluster {cluster_id}")
        return None

    source = ColumnDataSource({
        'Price': cluster_data['Price'].tolist(),
        'Category': cluster_data['Category'].tolist(),
        'Product Name': cluster_data['Product Name'].tolist()
    })

    categories = sorted(cluster_data['Category'].unique())
    
    plot = figure(plot_width=600, plot_height=400, title=f"Cluster {cluster_name.values[0]} - Price vs Category",
                  y_range=categories)
    plot.scatter(x='Price', y='Category', source=source, size=8)
    plot.xaxis.axis_label = "Price"
    plot.yaxis.axis_label = "Category"
    
    # Adding hover tool
    hover = HoverTool()
    hover.tooltips = """
        <div>
            <h3>@{Product Name}</h3>
            <div><strong>Price:</strong> @{Price}</div>
            <div><strong>Link:</strong> <a href="@{Product Link}" target="_blank">Click for product</a></div>
        </div>
    """
    plot.add_tools(hover)
    
    return plot

def update_plot(change):
    cluster_id = int(change.new)
    new_plot = create_cluster_plot(cluster_id, df)
    if new_plot:
        target.clear_output(wait=True)
        with target:
            show(new_plot, notebook_handle=True)

# Dropdown widget
dropdown = widgets.Dropdown(options=[str(i) for i in range(150)], description='Select Cluster:')
dropdown.observe(update_plot, names='value')

target = widgets.Output()

initial_plot = create_cluster_plot(0, df)
with target:
    show(initial_plot, notebook_handle=True)

display(dropdown, target)


Loading BokehJS ...

Dropdown(description='Select Cluster:', options=('0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11',…

Output()

# Update the visualization price analysis plot

In [139]:
from bokeh.models import CustomJS, TapTool, ColumnDataSource, HoverTool
from bokeh.plotting import figure
import pandas as pd

def create_cluster_plot(cluster_id, df):
    cluster_data = df[df['Cluster'] == cluster_id]
    cluster_name = df['Cluster_Name'][df['Cluster'] == cluster_id]

    if cluster_data.empty:
        print(f"No data found for cluster {cluster_id}")
        return None

    source = ColumnDataSource({
        'Price': cluster_data['Price'].tolist(),
        'Category': cluster_data['Category'].tolist(),
        'Product Name': cluster_data['Product Name'].tolist(),
        'Product Link': cluster_data['Product Link Cleaned'].tolist()  # Ensure this column exists
    })

    categories = sorted(cluster_data['Category'].unique())
    
    plot = figure(plot_width=600, plot_height=400, title=f"Cluster {cluster_name.values[0]} - Price vs Category",
                  y_range=categories)
    plot.scatter(x='Price', y='Category', source=source, size=8, color='navy', alpha=0.5)

    plot.xaxis.axis_label = "Price"
    plot.yaxis.axis_label = "Category"

    # Hover tool
    hover = HoverTool()
    hover.tooltips = """
        <div>
            <h3>@{Product Name}</h3>
            <div><strong>Price:</strong> @{Price}</div>
        </div>
    """
    plot.add_tools(hover)

    # JavaScript callback for clickable points
    code = """
        if (cb_data.source.selected.indices.length > 0){
            var selected_index = cb_data.source.selected.indices[0];
            var link = cb_data.source.data['Product Link'][selected_index];
            window.open(link);
        }
    """
    callback = CustomJS(args={'source': source}, code=code)
    plot.add_tools(TapTool(callback=callback))

    return plot

def update_plot(change):
    cluster_id = int(change.new)
    new_plot = create_cluster_plot(cluster_id, df)
    if new_plot:
        target.clear_output(wait=True)
        with target:
            show(new_plot, notebook_handle=True)

# Dropdown widget
dropdown = widgets.Dropdown(options=[str(i) for i in range(150)], description='Select Cluster:')
dropdown.observe(update_plot, names='value')

# Output widget for the plot
target = widgets.Output()

# default pick of cluster 0
initial_plot = create_cluster_plot(0, df)
with target:
    show(initial_plot, notebook_handle=True)

# Display the widgets
display(dropdown, target)



Dropdown(description='Select Cluster:', options=('0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11',…

Output()

In [48]:
df.to_csv('recommendation.csv')

In [54]:
df['Product Image Cleaned'] = df['Product Image'].str.replace("^'", '', regex=True)
df['Product Link Cleaned'] = df['Product Link'].str.replace("^'", '', regex=True)

In [79]:
# import ipywidgets as widgets
# from IPython.display import display, HTML
# import pandas as pd
# from sklearn.metrics.pairwise import cosine_similarity

# # Function to recommend products based on a search query
# def recommend_products(query, df, num_recommendations=5):
#     # Search for products by name similarity
#     keywords = query.lower().split()

#     # Create a new column for a simplified version of product names
#     df['Simplified Name'] = df['Product Name'].str.lower()

#     # Find products that match any of the keywords
#     keyword_matches = df[df['Simplified Name'].apply(lambda x: any(keyword in x for keyword in keywords))]

#     # If no direct keyword matches, use TF-IDF and cosine similarity for a broader search
#     if keyword_matches.empty:
#         tfidf_vectorizer = TfidfVectorizer()
#         tfidf_matrix = tfidf_vectorizer.fit_transform(df['Simplified Name'])

#         query_vec = tfidf_vectorizer.transform([query.lower()])
#         cosine_similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()

#         # Get the indices of the top matches
#         top_indices = np.argsort(cosine_similarities)[-num_recommendations:][::-1]

#         # Get the top matching products
#         recommendations = df.iloc[top_indices]
#     else:
#         recommendations = keyword_matches.sort_values(by='Price').head(num_recommendations)

#     # Drop the temporary column
#     df.drop(columns=['Simplified Name'], inplace=True)

#     return recommendations

# # Function to handle search
# def handle_search(sender):
#     query = search_box.value
#     recommendations = recommend_products(query, df)

#     # Clear previous output
#     output.clear_output()

#     # Display recommendations
#     with output:
#         if not recommendations.empty:
#             for _, row in recommendations.iterrows():
#                 display_html = f"""
#                     <div>
#                         <p><strong>Product:</strong> {row['Product Name']}<br>
#                            <strong>Price:</strong> {row['Price']}<br>
#                            <img src="{row['Product Image Cleaned']}" width="100"><br>
#                            <a href="{row['Product Link Cleaned']}" target="_blank">Product Link</a>
#                         </p>
#                     </div>
#                 """
#                 display(HTML(display_html))
#         else:
#             display(HTML("<div>No matching products found.</div>"))

# # Search box widget
# search_box = widgets.Text(description="Search:", placeholder='Enter product name or keyword')
# search_box.on_submit(handle_search)

# # Output widget for displaying results
# output = widgets.Output()

# # Display the search box and output area
# display(search_box, output)


Text(value='', description='Search:', placeholder='Enter product name or keyword')

Output()

In [140]:
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


# Stem text to avoid miss-matching
def stem_text(text):
    stemmer = PorterStemmer()
    words = nltk.word_tokenize(text)
    return ' '.join([stemmer.stem(word) for word in words])

# Now recommend products based on a search query
def recommend_products(query, df, plot_area, num_recommendations=5):
    
    query = stem_text(query.lower())
    
    name_similar = df[df['processed_Name'].str.contains(query, case=False, na=False)]

    # If no products are found, return an empty DataFrame
    if name_similar.empty:
        return pd.DataFrame()

    # Find the most common cluster among the matching products
    most_common_cluster = name_similar['Cluster'].mode()[0]

    # Update the plot for the most common cluster
    if most_common_cluster is not None:
        new_plot = create_cluster_plot(most_common_cluster, df)
        if new_plot:  # Check if new_plot is a valid plot object
            handle = show(new_plot, notebook_handle=True)
            plot_area.clear_output(wait=True)
            with plot_area:
                push_notebook(handle=handle)
        else:
            plot_area.clear_output()
            with plot_area:
                print("No plot available for this cluster.")
    else:
        plot_area.clear_output()
        with plot_area:
            print("No matching cluster found for the search query.")
            
    # Find products in the most common cluster but different categories
    cluster_products = df[(df['Cluster'] == most_common_cluster) & (~df['Category'].isin(name_similar['Category']))]
    
    # Sort by price and select the least expensive ones
    recommendations = cluster_products.sort_values(by='Price').head(num_recommendations)

    return recommendations

# Function to handle search
def handle_search(sender):
    query = search_box.value
    recommendations = recommend_products(query, df, plot_output)

    # Clear previous output
    output.clear_output()

    # Display recommendations
    with output:
        if not recommendations.empty:
            for _, row in recommendations.iterrows():
                display_html = f"""
                    <div>
                        <p><strong>Product:</strong> {row['Product Name']}<br>
                           <strong>Price:</strong> {row['Price']}<br>
                           <img src="{row['Product Image Cleaned']}" width="100"><br>
                           <a href="{row['Product Link Cleaned']}" target="_blank">Product Link</a>
                        </p>
                    </div>
                """
                display(HTML(display_html))
        else:
            display(HTML("<div>No matching products found.</div>"))

# Search box widget
search_box = widgets.Text(description="Search:", placeholder='Enter product name or keyword')
search_box.on_submit(handle_search)

# Output widget for displaying results (output as text, plot as price analysis)
output = widgets.Output()
plot_output = widgets.Output()

# Display the search box and output area
display(search_box, output, plot_output)


Text(value='', description='Search:', placeholder='Enter product name or keyword')

Output()

Output()